In [1]:
import tensorflow as tf
import numpy as np

tf.enable_eager_execution()

import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [2]:
impression_embed_size = 200 # 853,540개 
poi_embed_size = 50      # 13,352개
filter_embed_size = 10   # 208개
platform_embed_size = 5       # 55개
city_embed_size = 50          # 34,752개
sort_embed_size = 7 #interaction 빼고 one-hot
device_embed_size = 3 # mobile, desktop, tablet one-hot

In [3]:
# Parameters
learning_rate = 0.01
num_epoch = 10
batch_size = 1

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
n_hidden_3 = 128 # 3rd layer number of neurons

input_size = poi_embed_size+filter_embed_size+sort_embed_size+impression_embed_size+platform_embed_size+city_embed_size+\
device_embed_size+filter_embed_size + impression_embed_size + 2

X = tf.placeholder(tf.float32, [None, input_size])
Y = tf.placeholder(tf.float32, [None, 1])

In [4]:
# Define the neural network
def neural_net(XX_input):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.layers.dense(XX_input, n_hidden_1,activation=tf.nn.relu)
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.layers.dense(layer_1, n_hidden_2,activation=tf.nn.relu)
#     layer_2_dropout = tf.layers.dropout(layer2, training=True)
    # Output fully connected layer with a neuron for each class
    layer_3 = tf.layers.dense(layer_2, n_hidden_3, activation=tf.nn.relu)
    out_layer = tf.layers.dense(layer_3, 1, activation=tf.nn.sigmoid)
    return out_layer

In [5]:
import pandas as pd
import functions as f
from collections import Counter

In [6]:
train_df = pd.read_csv('./train.csv')
test_df = pd.read_csv('./test.csv')
meta_df = pd.read_csv('./item_metadata.csv')
submit_df = pd.read_csv('./submission_popular.csv')

popular_df = f.get_popularity(train_df)

In [7]:
poi_idx = np.load('./npy/poi_names.npy', allow_pickle=True)
poi_idx = list(poi_idx)
impressions_idx = np.load('./npy/impressions_index.npy', allow_pickle=True)
impressions_idx = list(impressions_idx)
city_idx = np.load('./npy/city_names.npy', allow_pickle=True)
city_idx = list(city_idx)
platform_idx = np.load('./npy/platform_names.npy', allow_pickle=True)
platform_idx = list(platform_idx)
filter_idx = np.load('./npy/filter_merged.npy', allow_pickle=True)
filter_idx = list(filter_idx)
action_idx = np.load('./npy/action_type_names.npy', allow_pickle=True)
action_idx = list(action_idx)

sort_order_idx = np.load('./npy/sorting_names.npy', allow_pickle=True)
sort_order_idx = list(sort_order_idx)
device_idx = np.load('./npy/device_names.npy', allow_pickle=True)
device_idx = list(device_idx)

poi_embedding = tf.get_variable("poi_embedding", [len(poi_idx), poi_embed_size])
impression_embedding = tf.get_variable("impression_embedding", [len(impressions_idx), impression_embed_size])
city_embedding = tf.get_variable("city_embedding", [len(city_idx), city_embed_size])
platform_embedding = tf.get_variable("platform_embedding", [len(platform_idx), platform_embed_size])
filter_embedding = tf.get_variable("filter_embedding", [len(filter_idx), filter_embed_size])

Instructions for updating:
Colocations handled automatically by placer.


In [8]:
train_label = train_df[(train_df["action_type"] == "clickout item")]     #train_label : (1,586,586, 12)
batch_indexes = np.random.permutation(len(train_label))

In [9]:
loss = tf.nn.sigmoid_cross_entropy_with_logits(labels= Y, logits= neural_net(X))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

Instructions for updating:
Use keras.layers.dense instead.


In [41]:
def embedding_function(domain, key):
    
    embedded_vector = None
    
    if domain == 'poi':
        if key in poi_idx:
            idx = poi_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(poi_embedding, idx)
        
    elif domain == 'impression':
        if key in impressions_idx:
            idx = impressions_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(impression_embedding, idx)
    
    elif domain == 'city':
        if key in city_idx:
            idx = city_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(city_embedding, idx)

    elif domain == 'platform':
        if key in platform_idx:
            idx = platform_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(platform_embedding, idx)
        
    elif domain =='filter':
        if key in filter_idx:
            idx = filter_idx.index(key)
        else:
            idx = 0
        embedded_vector = tf.nn.embedding_lookup(filter_embedding, idx)
    elif domain =='sorting':
        embedded_vector = np.zeros([sort_embed_size])
        embedded_vector[sort_order_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
    elif domain =='device':
        embedded_vector = np.zeros([device_embed_size])
        embedded_vector[device_idx.index(key)] = 1
        embedded_vector = tf.convert_to_tensor(embedded_vector, dtype=tf.float32)
        
    return embedded_vector

In [11]:
 # weighted sum. 자주 등장한 keyword의 embedding을 더 따르도록
def getWeightedAverage(domain, domain_list, embed_size): 
    final_embed = np.zeros([embed_size])
    count_dic = Counter(domain_list)
    count_sum=0
    for key in count_dic:
        embed = embedding_function(domain, key)
        final_embed += count_dic[key] * embed  
        count_sum += count_dic[key]
    
    final_embed = final_embed/count_sum
    
    return final_embed

In [53]:
def getTrainData(train_x_df, train_label_batch):
    
    # poi 
    poi_list = train_x_df[(train_x_df['action_type']=='search for poi')]['reference']
    if len(poi_list)==0:
        final_embed_poi = tf.zeros([poi_embed_size])
    else:
        final_embed_poi = getWeightedAverage('poi', poi_list , poi_embed_size)
    
    
    #filters (current filters 총합)
    filter_list = train_x_df['current_filters'].dropna()
    if len(filter_list)==0:
        final_embed_filters = tf.zeros([filter_embed_size])
    else:
        filters = []
        for item in filter_list:
            item_split = f.string_to_array(item)
            filters = np.concatenate((filters, item_split))
        final_embed_filters = getWeightedAverage('filter', filters , filter_embed_size)
    
    #sort order
    sort_orders = train_x_df[(train_x_df['action_type']=='change of sort order') & (train_x_df['reference']!='interaction sort button')]["reference"]
    if len(sort_orders)==0:
        final_embed_sorting = tf.zeros([sort_embed_size])
    else:
        sort_list=[]
        for item in sort_orders:
            sort_list.append(item)
        final_embed_sorting = getWeightedAverage('sorting', sort_list , sort_embed_size)
    
    # 관심있는 item
            #  'clickout item'
            # 'interaction item deals',
            #  'interaction item image',
            #  'interaction item info',
            #  'interaction item rating',
            # 'search for item',
    item_list = train_x_df[(train_x_df['action_type']=='clickout item')|(train_x_df['action_type']=='interaction item deals')
                      |(train_x_df['action_type']=='interaction item image')|(train_x_df['action_type']=='interaction item info')
                      |(train_x_df['action_type']=='interaction item rating')|(train_x_df['action_type']=='search for item')]['reference']
    
    if len(item_list)==0:
        final_embed_items = tf.zeros([impression_embed_size])
    else:
        item_merged_list=[]
        for item in item_list:
            item_merged_list.append(item)
        final_embed_items = getWeightedAverage('impression', item_merged_list , impression_embed_size)
    
    #platform
    platform_embed = embedding_function('platform', train_label_batch['platform'])
    
    #city, search for destination 해서 검색해봤던 city들
    city_embed = embedding_function('city', train_label_batch['city'])
    
    #device
    device_embed = embedding_function('device', train_label_batch['device'])
    
    #current_filter
    
    filter_b = train_label_batch['current_filters']
    if pd.isna(filter_b):
        final_embed_current_filter = tf.zeros(filter_embed_size)
    else:
        filters_b = f.string_to_array(filter_b)
        final_embed_current_filter = getWeightedAverage('filter', filters_b , filter_embed_size)
    
#     print('poi : ', np.shape(final_embed_poi))
#     print('filters : ', np.shape(final_embed_filters))
#     print('sorting : ', np.shape(final_embed_sorting))
#     print('items : ', np.shape(final_embed_items))
#     print('platform : ', np.shape(platform_embed))
#     print('city : ', np.shape(city_embed))
#     print('device : ', np.shape(device_embed))
#     print('current filter : ', np.shape(final_embed_current_filter))
    
    
    return tf.concat((final_embed_poi, final_embed_filters, final_embed_sorting, final_embed_items, platform_embed, city_embed, device_embed, final_embed_current_filter), axis=0)

In [55]:
label_df = None
for k in range(3):
    idx = batch_indexes[k*batch_size:(k+1)*batch_size]
    label_df = train_label.iloc[idx]
    current_user = label_df["user_id"]
    for i in range(3):  # batch size
        train_df_current_user = train_df[train_df["user_id"]==current_user.iloc[i]]
        train_idx = train_df_current_user.index.get_loc(label_df.iloc[i].name)
        train_df_trucated = train_df_current_user[:train_idx]
        
        if len(train_df_trucated)>30:
            train_df_trucated = train_df_trucated[-30:]
        
        train_label_batch = label_df.iloc[i]["reference"]  # 클릭한 reference number
        train_candidates = label_df.iloc[i]['impressions']
        train_candidates = f.string_to_array(train_candidates)   
        
        candidate_prices = label_df.iloc[i]['prices']
        candidate_prices = f.string_to_array(candidate_prices)
        candidate_prices = np.array(candidate_prices).astype(int)
        # train_candidates 중에 정답을 고르게 된다. 
        # 학습 할 때는 선택 안한 애는 라벨 0 , 선택한 애는 라벨 1. infer할 때는 0.9 인 애를 고르면 됨.
        context_vector = getTrainData(train_df_trucated, label_df.iloc[i])  # 현재 335 크기 context vector
        
        real_labels = np.zeros([len(train_candidates)])
        real_labels[train_candidates.index(train_label_batch)] = 1
        
        for h in range(len(train_candidates)):
            candi = train_candidates[h]
            price_candi = candidate_prices[h]
            candidate_vector = embedding_function('impression', candi)
            
            pop_temp = popular_df[popular_df['reference']==candi]
            if len(pop_temp)==0:
                popularity= 0
            else: 
                popularity = int(popular_df[popular_df['reference']==candi]['n_clicks'])
                if popularity > 10:
                    popularity = 10
            popular_vector = (popularity - 0)/(10-0)
            
            if price_candi < 15:
                price_candi = 15
            elif price_candi > 325:
                price_candi = 325
            price_vector = (price_candi - 15)/(325-15)
            XX = tf.concat((context_vector, candidate_vector, tf.convert_to_tensor(np.array([popular_vector, price_vector]),dtype=tf.float32)),axis=0)
            YY = real_labels[h]
            _, loss_val = sess.run([optimizer, loss], feed_dict={X: XX, Y: YY})
            print("i: ", i ,", h:", h, "loss:", loss_val)

poi :  (50,)
filters :  (10,)
sorting :  (7,)
items :  (200,)
platform :  (5,)
city :  (50,)
device :  (3,)
current filter :  (10,)


/home/ecube_server2/.conda/envs/recsys/lib/python3.6/site-packages/pandas/core/ops.py:1649: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


TypeError: The value of a feed cannot be a tf.Tensor object. Acceptable feed values include Python scalars, strings, lists, numpy ndarrays, or TensorHandles. For reference, the tensor object was Tensor("concat_3:0", shape=(537,), dtype=float32) which was passed to the feed with key Tensor("Placeholder:0", shape=(?, 537), dtype=float32).

In [ ]:
# merged = np.array([],dtype=int)

# for i in range(len(impression_total)):
#     impressions = impression_total.iloc[i]["impressions"]
#     s = f.string_to_array(impressions)
#     xx = np.array(s)
#     xx = xx.astype(int)
#     merged = np.concatenate((merged, xx))
#     if i % 10000 ==0:
#         print("iter : ", i, "len:" ,len(merged))
#         merged = np.unique(merged)
#         print("merged : ", len(merged))